In [1]:

import pandas as pd
from lxml import etree
import io
import json
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import hamming_loss
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import re
import os


def convert_to_adage_json(df, dataset_id):
    adage_data_model = {
        "data_source": "Australian Financial Review",
        "dataset_type": "News_Articles",
        "dataset_id": dataset_id,
        "time_object": {
            "timestamp": pd.Timestamp.now().isoformat(),
            "timezone": "GMT+11"
        },
        "events": []
    }
    
    for index, row in df.iterrows():
        event = {
            "time_object": {
                "timestamp": row["modified"].isoformat(),
                "duration": 0,
                "duration_unit": "second",
                "timezone": "GMT+11"
            },
            "event_type": "article",
            "attribute": {
                "guid": row["guid"],
                "byline": row["byline"],
                "headline": row["headline"],
                "section": row["section"],
                "publication_date": row["publication_date"].strftime("%Y-%m-%d"),
                "page_no": row["page_no"],
                "classifications": row["classifications"],
                "text": row.get("text")
            }
        }
        
        adage_data_model["events"].append(event)
    
    adage_data_model["time_object"]["timestamp"] = df["modified"].max().isoformat()
    
    return json.dumps(adage_data_model, indent=4)

def process_xml_file(file_path):
    xml_data = open(file_path).read()
    parser = etree.XMLParser(ns_clean=True)
    xml = etree.parse(io.StringIO(xml_data), parser)
    data = []

    for dossier in xml.xpath('//dcdossier'):
        guid = dossier.get('guid')
        modified = dossier.get('modified')
        
        for doc in dossier.xpath('.//document'):
            newspaper_code = doc.xpath('.//NEWSPAPERCODE/text()')
            section = doc.xpath('.//SECTION/text()')
            story_name = doc.xpath('.//STORYNAME/text()')
            publication_date = doc.xpath('.//PUBLICATIONDATE/text()')
            newspaper = doc.xpath('.//NEWSPAPER/text()')
            page_no = doc.xpath('.//PAGENO/text()')
            byline = doc.xpath('.//BYLINE/text()')
            classifications = doc.xpath('.//CLASSIFICATION/text()')
            headline = doc.xpath('.//HEADLINE/text()')
            intro = doc.xpath('.//INTRO/text()')
            text = " ".join(doc.xpath('.//TEXT//text()'))
            
            data.append({
                'guid': guid,
                'modified': pd.to_datetime(modified, errors='coerce', utc=True),
                'section': section[0].strip() if section else None,
                'publication_date': pd.to_datetime(publication_date[0]) if publication_date else None,
                'page_no': page_no[0].strip() if page_no else None,
                'byline': byline[0].strip() if byline else None,
                'classifications': classifications if classifications else None,
                'headline': headline[0].strip() if headline else None,
                'intro': intro[0].strip() if intro else None,
                'text': text.strip() if text else None,
            })

    return pd.DataFrame(data)


def process_all_files(directory):
    all_dataframes = []
    j = 0
    for file_name in os.listdir(directory):
        if file_name.endswith('.xml'):
            file_path = os.path.join(directory, file_name)
            df = process_xml_file(file_path)
            all_dataframes.append(df)
            print(f"Processed data from {file_name}")
            j += 1
        if j == 10: 
            break

    combined_df = pd.concat(all_dataframes, ignore_index=True)
    return combined_df


df = process_all_files('datasets/')
# df = pd.DataFrame(data)
df['modified'] = pd.to_datetime(df['modified'])
df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce')


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk_stopwords = stopwords.words('english')


def clean_and_prepare_text(input_text, pattern=r'[^\w\s]', to_lowercase=True, filter_stopwords=True, stemming=True):
    cleaned_text = re.sub(pattern, '', input_text)
    if to_lowercase:
        cleaned_text = cleaned_text.lower()
    tokenized_text = word_tokenize(cleaned_text)
    if filter_stopwords:
        tokenized_text = [word for word in tokenized_text if word not in nltk_stopwords]
    if stemming:
        processor = PorterStemmer()
    else:
        processor = WordNetLemmatizer()
    processed_text = [processor.stem(word) if stemming else processor.lemmatize(word) for word in tokenized_text]
    return ' '.join(processed_text)


preprocess_settings = (r'[^\w\s]', False, True, False)
pattern, lower, stopword_removal, stem = preprocess_settings
preprocess_fn = lambda x: clean_and_prepare_text(x, pattern, lower, stopword_removal, stem)
df['pre_processed_text'] = df['text'].apply(preprocess_fn)

Processed data from AFR_20150101-20150131.xml
Processed data from AFR_20150201-20150201.xml
Processed data from AFR_20150201-20150228.xml
Processed data from AFR_20150301-20150331.xml
Processed data from AFR_20150401-20150430.xml
Processed data from AFR_20150501-20150531.xml
Processed data from AFR_20150601-20150630.xml
Processed data from AFR_20150701-20150731.xml
Processed data from AFR_20150801-20150831.xml
Processed data from AFR_20150901-20150930.xml


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ricardo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ricardo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ricardo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df

In [ ]:
from collections import Counter

def split_labels(label_list):
    if label_list is None:
        return []
    split_labels = []
    for label in label_list:
        parts = label.split('/', maxsplit=1)
        split_labels.append(parts[0])
        if len(parts) > 1:
            split_labels.append(parts[1])
    return split_labels

all_labels = []
for labels in df['classifications']:
    all_labels.extend(split_labels(labels))

label_counts = Counter(all_labels)
top_labels = label_counts.most_common(500)

print("Top 100 labels:")
for label, count in top_labels:
    print(f"{label}: {count}")
    
    
    


In [5]:
# Business and Economy
# Politics and Government
# Finance and Markets
# Technology and Internet
# Property and Real Estate
# Employment and Workforce
# Energy and Resources
# Media and Entertainment
# Law and Crime
# Education and Research
# Healthcare and Pharmaceuticals
# Infrastructure and Transport
# Environment and Sustainability
# Society and Culture
# International Relations and Trade

In [ ]:
from gensim import corpora, models
from gensim.models import CoherenceModel

df['tokenized_text'] = df['pre_processed_text'].apply(lambda x: x.split())

stop_words = set(stopwords.words('english'))
df['tokenized_text'] = df['tokenized_text'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

dictionary = corpora.Dictionary(df['tokenized_text'])

corpus = [dictionary.doc2bow(text) for text in df['tokenized_text']]

num_topics_range = range(5, 21, 5)
coherence_scores = []

for num_topics in num_topics_range:
    lda_model = models.LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=10, workers=4)
    
    coherence_model = CoherenceModel(model=lda_model, texts=df['tokenized_text'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append(coherence_score)
    
    print(f"Number of Topics: {num_topics}")
    print("Topic Keywords:")
    for topic in lda_model.print_topics(num_words=10):
        print(topic)
    print("Coherence Score:", coherence_score)
    print()


best_num_topics = num_topics_range[coherence_scores.index(max(coherence_scores))]
print("Best Number of Topics:", best_num_topics)


In [9]:
topic_words = { "Business and Economy": [
    "superannuation", "gst", "asic", "ato", "bas", "payg", "fringe", "stamp", "land", "duty",
    "mining", "agriculture", "tourism", "education", "finance", "construction", "manufacturing", "retail",
    "health", "services", "technology", "innovation", "startup", "export", "import", "trade",
    "investment", "equity", "currency", "dollar", "shares", "stock", "bond", "commodity",
    "market", "exchange", "sydney", "melbourne", "brisbane", "perth", "adelaide", "canberra",
    "hobart", "darwin", "rba", "interest", "rate", "loan", "mortgage", "banking",
    "insurance", "pension", "superfund", "dividend", "revenue", "profit", "loss", "budget",
    "deficit", "surplus", "inflation", "gdp", "economy", "fiscal", "monetary", "policy",
    "regulation", "deregulation", "tariff", "quota", "barrier", "agreement", "partnership", "competition",
    "monopoly", "oligopoly", "sme", "enterprise", "corporation", "subsidiary", "branch", "headquarters",
    "tax", "credit", "debit", "ledger", "accounting", "audit", "balance", "statement",
    "cashflow", "asset", "liability", "equity", "valuation", "merger", "acquisition", "ipo",
    "venture", "capital", "angel", "investor", "crowdfunding", "portfolio", "diversification", "risk",
    "return", "yield", "benchmark", "index", "compliance", "governance", "ethics", "sustainability",
    "responsibility", "innovation", "disruption", "automation", "blockchain", "cryptocurrency", "fintech", "ai",
    "big", "data", "cloud", "computing", "e-commerce", "digital", "marketing", "social",
    "media", "branding", "consumer", "demand", "supply", "price", "cost", "wage",
    "salary", "employment", "unemployment", "labour", "force", "union", "negotiation", "contract",
    "legislation", "policy", "reform", "subsidy", "grant", "incentive", "trade", "deal"
], 
    "Politics and Government" : [
    "parliament", "senate", "house", "representatives", "government", "opposition", "coalition", "labor",
    "liberal", "greens", "nationals", "democracy", "federal", "state", "local", "election",
    "vote", "ballot", "campaign", "policy", "legislation", "bill", "law", "act",
    "constitution", "referendum", "plebiscite", "budget", "tax", "spending", "defence", "health",
    "education", "welfare", "infrastructure", "transport", "energy", "environment", "trade", "foreign",
    "affairs", "immigration", "citizenship", "rights", "freedom", "justice", "court", "high",
    "judiciary", "legal", "mp", "senator", "minister", "prime", "premier", "mayor",
    "council", "electorate", "constituency", "debate", "discussion", "policy", "reform", "amendment",
    "motion", "question", "session", "sitting", "committee", "inquiry", "report", "findings",
    "recommendation", "implementation", "regulation", "compliance", "oversight", "audit", "expenditure", "revenue",
    "deficit", "surplus", "debt", "borrowing", "funding", "grant", "subsidy", "pension",
    "benefit", "allowance", "service", "program", "initiative", "project", "partnership", "agreement",
    "treaty", "diplomacy", "security", "defence", "military", "intelligence", "surveillance", "border",
    "control", "asylum", "refugee", "settlement", "integration", "multiculturalism", "diversity", "equity",
    "inclusion", "gender", "equality", "access", "participation", "engagement", "transparency", "accountability",
    "ethics", "integrity", "corruption", "scandal", "investigation", "prosecution", "conviction", "sentencing",
    "appeal", "pardon", "indigenous", "aboriginal", "torres", "strait", "islander", "land",
    "rights", "treaty", "reconciliation", "sovereignty", "self-determination", "culture", "heritage", "preservation",
    "conservation", "climate", "change", "sustainability", "renewable", "energy", "carbon", "emission",
    "reduction", "adaptation", "mitigation", "biodiversity", "ecosystem", "protection", "management", "policy",
    "strategy", "planning", "development", "growth", "innovation", "technology", "research", "education",
    "healthcare", "social", "welfare", "community", "service", "support", "assistance", "relief"
], 
    "Finance and Markets": [
    "superannuation", "asic", "rba", "asx", "aud", "shares", "bonds", "dividends", "interest", "loans",
    "mortgages", "insurance", "banks", "fintech", "cryptocurrency", "blockchain", "investments", "portfolio",
    "superfund", "pension", "annuity", "taxation", "gst", "capital", "gains", "losses", "deductions",
    "budget", "deficit", "surplus", "economy", "inflation", "deflation", "recession", "growth",
    "gdp", "trade", "balance", "exports", "imports", "tariffs", "quotas", "foreign",
    "exchange", "currency", "valuation", "risk", "management", "hedge", "fund", "equity",
    "market", "liquidity", "volatility", "speculation", "arbitrage", "leverage", "margin", "broker",
    "dealer", "trader", "analyst", "adviser", "consultant", "audit", "compliance", "regulation",
    "legislation", "ethics", "governance", "transparency", "accountability", "credit", "rating", "debt",
    "securities", "derivative", "option", "futures", "swap", "commodity", "metal", "gold",
    "silver", "coal", "iron", "ore", "agriculture", "wheat", "wool", "cattle",
    "dairy", "water", "resources", "sustainable", "investment", "ethical", "green", "bonds",
    "climate", "change", "carbon", "emissions", "renewable", "energy", "solar", "wind",
    "technology", "innovation", "startup", "venture", "capital", "private", "equity", "ipo",
    "merger", "acquisition", "takeover", "restructuring", "bankruptcy", "insolvency", "payout", "return",
    "yield", "diversification", "strategy", "planning", "forecasting", "analysis", "research", "report",
    "data", "statistics", "benchmark", "index", "fund", "management", "fee", "expense",
    "income", "wealth", "savings", "retirement", "estate", "planning", "trust", "beneficiary"
],  
    "Technology and Internet": [
    "nbn", "broadband", "telstra", "optus", "vodafone", "5g", "internet", "wifi", "cloud", "data",
    "security", "encryption", "privacy", "vpn", "ai", "machine", "learning", "robotics", "automation", "software",
    "hardware", "device", "mobile", "smartphone", "tablet", "laptop", "desktop", "server", "storage", "network",
    "infrastructure", "technology", "innovation", "startup", "entrepreneur", "venture", "capital", "funding", "research",
    "development", "digital", "transformation", "e-commerce", "online", "shopping", "payment", "gateway", "fintech",
    "cryptocurrency", "blockchain", "bitcoin", "ethereum", "cybersecurity", "hacking", "phishing", "malware", "ransomware",
    "firewall", "antivirus", "software", "development", "programming", "coding", "algorithm", "application", "app",
    "platform", "user", "interface", "experience", "design", "web", "website", "hosting", "domain", "social",
    "media", "facebook", "twitter", "instagram", "linkedin", "youtube", "google", "search", "engine", "optimization",
    "analytics", "data", "mining", "big", "technology", "trends", "virtual", "reality", "augmented", "reality",
    "iot", "internet", "things", "wearable", "technology", "health", "tech", "edtech", "legaltech", "agtech",
    "energy", "sustainability", "green", "tech", "cloud", "computing", "saas", "paas", "iaas", "subscription",
    "model", "scalability", "agile", "scrum", "devops", "cyber", "safety", "ethics", "regulation", "policy",
    "compliance", "gdpr", "accc", "afr", "ato", "asx", "asic", "innovation", "hub", "incubator", "accelerator"
],
    "Property and Real Estate" : [
    "property", "real", "estate", "housing", "market", "mortgage", "rent", "lease", "landlord", "tenant",
    "apartment", "house", "townhouse", "villa", "unit", "estate", "agent", "broker", "auction", "sale",
    "purchase", "offer", "bid", "contract", "settlement", "conveyancing", "title", "deed", "stamp", "duty",
    "valuation", "appraisal", "inspection", "renovation", "development", "planning", "zoning", "council", "permit",
    "construction", "building", "architecture", "design", "landscape", "garden", "pool", "garage", "driveway",
    "fence", "security", "alarm", "cctv", "energy", "efficiency", "solar", "panels", "water", "tank",
    "heating", "cooling", "air", "conditioning", "insulation", "location", "suburb", "city", "rural", "coastal",
    "investment", "portfolio", "yield", "return", "capital", "gain", "loss", "equity", "leverage", "finance",
    "bank", "lender", "interest", "rate", "fixed", "variable", "deposit", "downpayment", "savings", "budget",
    "insurance", "policy", "claim", "coverage", "risk", "survey", "report", "maintenance", "repair", "upgrade",
    "fixture", "fitting", "furniture", "decor", "style", "trend", "market", "analysis", "forecast", "trend",
    "bubble", "crash", "boom", "slump", "recovery", "affordability", "accessibility", "first", "home", "buyer",
    "incentive", "scheme", "grant", "stamp", "duty", "concession", "negative", "gearing", "capital", "works",
    "depreciation", "tax", "break", "benefit", "exemption", "reit", "trust", "portfolio", "diversification"
],
    "Employment and Workforce": [
    "employment", "workforce", "job", "career", "occupation", "profession", "industry", "sector", "company", "business",
    "organisation", "employer", "employee", "workplace", "office", "factory", "site", "shift", "hour", "wage",
    "salary", "income", "pay", "remuneration", "benefit", "bonus", "overtime", "leave", "holiday", "vacation",
    "sick", "maternity", "paternity", "flexible", "part-time", "full-time", "casual", "contract", "temporary", "permanent",
    "union", "negotiation", "agreement", "contract", "dispute", "strike", "lockout", "mediation", "arbitration", "award",
    "commission", "fair", "work", "standards", "rights", "entitlement", "dismissal", "redundancy", "termination", "resignation",
    "recruitment", "hiring", "application", "interview", "cv", "resume", "cover", "letter", "reference", "background",
    "check", "skill", "qualification", "experience", "training", "development", "promotion", "advancement", "career", "path",
    "succession", "planning", "mentorship", "coaching", "performance", "review", "feedback", "goal", "objective", "assessment",
    "diversity", "inclusion", "equality", "equity", "culture", "engagement", "wellbeing", "safety", "health", "work-life",
    "balance", "telework", "remote", "work", "home", "office", "commute", "mobility", "relocation", "expatriate",
    "immigration", "visa", "skill", "shortage", "talent", "acquisition", "retention", "turnover", "attrition", "layoff",
    "outsourcing", "offshoring", "gig", "economy", "freelance", "contractor", "self-employed", "entrepreneur", "startup", "innovation"
], 
    "Energy and Resources": [
    "coal", "iron", "ore", "gold", "copper", "bauxite", "uranium", "nickel", "gas", "natural",
    "petroleum", "oil", "shale", "renewable", "solar", "wind", "hydro", "geothermal", "biomass", "biofuel",
    "energy", "power", "electricity", "generation", "transmission", "distribution", "grid", "storage", "battery", "tesla",
    "efficiency", "sustainability", "carbon", "emission", "reduction", "climate", "change", "environment", "conservation", "water",
    "management", "waste", "recycling", "policy", "regulation", "legislation", "subsidy", "tariff", "incentive", "investment",
    "project", "development", "exploration", "extraction", "mining", "drilling", "fracking", "export", "import", "market",
    "price", "demand", "supply", "trade", "agreement", "partnership", "technology", "innovation", "research", "development",
    "solar", "panel", "turbine", "farm", "hydroelectric", "dam", "nuclear", "fission", "fusion", "reactor",
    "ethanol", "biodiesel", "liquefied", "compressed", "natural", "gas", "lng", "cng", "pipeline", "infrastructure",
    "network", "capacity", "peak", "off-peak", "baseload", "dispatchable", "intermittent", "reliability", "security", "vulnerability",
    "resilience", "adaptation", "mitigation", "audit", "assessment", "benchmark", "certification", "standard", "rating", "label",
    "footprint", "offset", "sequestration", "capture", "storage", "leakage", "vent", "flare", "rehabilitation", "remediation",
    "safeguard", "protection", "conservation", "stewardship", "custodian", "heritage", "landscape", "biodiversity", "ecosystem", "habitat"
], 
    "Media and Entertainment": [
    "television", "radio", "newspaper", "magazine", "journalism", "broadcast", "streaming", "podcast", "film", "cinema",
    "documentary", "series", "drama", "comedy", "reality", "news", "sport", "music", "concert", "festival",
    "theatre", "play", "performance", "art", "exhibition", "gallery", "book", "novel", "literature", "poetry",
    "dance", "ballet", "opera", "symphony", "orchestra", "band", "singer", "musician", "composer", "dj",
    "video", "game", "gaming", "esports", "virtual", "reality", "animation", "graphic", "design", "photography",
    "cinematography", "production", "editing", "soundtrack", "score", "album", "track", "stream", "download", "platform",
    "social", "media", "facebook", "twitter", "instagram", "youtube", "tiktok", "snapchat", "linkedin", "blog",
    "vlog", "influencer", "content", "creator", "advertisement", "marketing", "promotion", "campaign", "branding", "sponsorship",
    "audience", "viewer", "listener", "reader", "subscriber", "rating", "review", "critic", "award", "nomination",
    "fame", "celebrity", "star", "talent", "agent", "manager", "producer", "director", "screenwriter", "actor",
    "actress", "host", "presenter", "reporter", "anchor", "journalist", "editor", "publisher", "copyright", "license",
    "royalty", "streaming", "service", "app", "website", "channel", "network", "satellite", "cable", "digital",
    "analog", "hd", "4k", "vr", "ar", "ai", "tech", "innovation", "trend", "future", "culture", "entertainment",
    "leisure", "hobby", "interest", "passion", "creativity", "inspiration", "imagination", "story", "narrative", "character"
], 
    "Law and Crime": [
    "law", "crime", "justice", "court", "trial", "judge", "jury", "defendant", "plaintiff", "prosecution",
    "defense", "verdict", "sentence", "appeal", "bail", "custody", "prison", "parole", "lawyer", "solicitor",
    "barrister", "attorney", "legal", "litigation", "suit", "case", "evidence", "testimony", "witness", "allegation",
    "charge", "conviction", "acquittal", "punishment", "fine", "community", "service", "rehabilitation", "crime", "rate",
    "theft", "robbery", "burglary", "assault", "fraud", "embezzlement", "corruption", "bribery", "money", "laundering",
    "drug", "trafficking", "homicide", "manslaughter", "kidnapping", "arson", "vandalism", "trespass", "cybercrime", "phishing",
    "hacking", "identity", "theft", "piracy", "stalking", "harassment", "assault", "battery", "domestic", "violence",
    "sexual", "assault", "rape", "child", "abuse", "neglect", "smuggling", "terrorism", "extortion", "blackmail",
    "illegal", "gambling", "prostitution", "human", "trafficking", "counterfeiting", "forgery", "perjury", "obstruction", "justice",
    "conspiracy", "manslaughter", "negligence", "malpractice", "infringement", "libel", "slander", "defamation", "privacy", "breach",
    "discrimination", "rights", "freedom", "constitution", "legislation", "regulation", "ordinance", "statute", "code", "act",
    "amendment", "policy", "procedure", "enforcement", "agency", "police", "detective", "officer", "sheriff", "marshal",
    "federal", "state", "local", "government", "authority", "jurisdiction", "sovereignty", "treaty", "agreement", "convention"
],
    "Education and Research": [
    "education", "research", "university", "college", "school", "academy", "institute", "scholarship", "tuition", "course",
    "curriculum", "program", "degree", "diploma", "certificate", "graduate", "undergraduate", "postgraduate", "phd", "doctorate",
    "faculty", "professor", "lecturer", "teacher", "student", "pupil", "enrollment", "learning", "study", "homework",
    "assignment", "thesis", "dissertation", "textbook", "library", "laboratory", "experiment", "science", "mathematics", "engineering",
    "technology", "humanities", "arts", "social", "sciences", "business", "law", "medicine", "nursing", "pharmacy",
    "biology", "chemistry", "physics", "geology", "psychology", "history", "geography", "language", "literature", "philosophy",
    "education", "policy", "reform", "accreditation", "assessment", "evaluation", "qualification", "standard", "achievement", "performance",
    "literacy", "numeracy", "skill", "competency", "training", "development", "workshop", "seminar", "conference", "symposium",
    "grant", "funding", "sponsorship", "endowment", "publication", "journal", "article", "paper", "findings", "results",
    "data", "analysis", "methodology", "ethics", "peer", "review", "collaboration", "partnership", "innovation", "breakthrough",
    "technology", "transfer", "patent", "copyright", "intellectual", "property", "discovery", "invention", "experimentation", "exploration",
    "fieldwork", "survey", "questionnaire", "interview", "case", "study", "research", "project", "thesis", "dissertation",
    "academia", "scholar", "scientist", "researcher", "fellow", "postdoc", "lectureship", "tenure", "curriculum", "pedagogy",
    "elearning", "online", "distance", "education", "mooc", "tutorial", "webinar", "virtual", "classroom", "platform"
], 
    "Healthcare and Pharmaceuticals": [
    "healthcare", "pharmaceuticals", "medicine", "hospital", "clinic", "surgery", "doctor", "nurse", "pharmacist", "dentist",
    "psychiatrist", "therapist", "patient", "treatment", "prescription", "drug", "medication", "vaccine", "therapy", "diagnosis",
    "illness", "disease", "condition", "symptom", "recovery", "rehabilitation", "surgical", "procedure", "operation", "emergency",
    "ambulance", "paramedic", "care", "support", "health", "wellness", "nutrition", "diet", "exercise", "fitness",
    "mental", "wellbeing", "counseling", "psychology", "research", "trial", "study", "laboratory", "innovation", "development",
    "biotechnology", "genetics", "genomics", "immunology", "oncology", "cardiology", "neurology", "pediatrics", "geriatrics", "endocrinology",
    "radiology", "imaging", "ultrasound", "mri", "ct", "scan", "vaccination", "immunization", "health", "insurance", "medicare",
    "bulk", "billing", "pharmacy", "dispensary", "formulary", "generic", "brand", "patent", "regulation", "compliance",
    "safety", "quality", "efficacy", "side", "effect", "adverse", "reaction", "contraindication", "dosage", "administration",
    "oral", "topical", "injectable", "infusion", "screening", "prevention", "public", "health", "epidemic", "pandemic",
    "outbreak", "contagion", "quarantine", "isolation", "hygiene", "sanitation", "virus", "bacteria", "infection", "antibiotic",
    "resistance", "immunity", "pathogen", "vector", "transmission", "vulnerability", "risk", "assessment", "management", "policy",
    "legislation", "ethics", "consent", "privacy", "data", "record", "electronic", "health", "record", "ehealth",
    "telehealth", "telemedicine", "digital", "health", "app", "platform", "wearable", "device", "monitoring", "intervention"
],
    "Infrastructure and Transport": [
    "infrastructure", "transport", "road", "highway", "freeway", "bridge", "tunnel", "rail", "train", "tram",
    "metro", "bus", "bicycle", "pathway", "pedestrian", "crossing", "airport", "runway", "hangar", "terminal",
    "port", "harbour", "dock", "wharf", "shipping", "cargo", "freight", "logistics", "distribution", "supply",
    "chain", "vehicle", "car", "truck", "van", "motorcycle", "scooter", "bike", "electric", "hybrid",
    "fuel", "diesel", "petrol", "charging", "station", "parking", "lot", "garage", "toll", "ticket",
    "fare", "pass", "card", "navigation", "gps", "traffic", "congestion", "signal", "sign", "lane",
    "roundabout", "intersection", "bypass", "public", "private", "commute", "journey", "trip", "route", "map",
    "engineering", "construction", "maintenance", "repair", "upgrade", "expansion", "investment", "project", "planning", "design",
    "safety", "security", "emergency", "service", "rescue", "evacuation", "accessibility", "mobility", "innovation", "technology",
    "sustainability", "environment", "emission", "pollution", "clean", "green", "renewable", "energy", "efficiency", "conservation",
    "policy", "regulation", "legislation", "authority", "agency", "department", "funding", "grant", "subsidy", "partnership",
    "network", "system", "capacity", "demand", "supply", "user", "customer", "passenger", "operator", "provider"
],
    "Environment and Sustainability": [
    "environment", "sustainability", "conservation", "biodiversity", "ecology", "nature", "wildlife", "habitat", "forest", "bushland",
    "reef", "ocean", "water", "river", "lake", "wetland", "coast", "beach", "desert", "outback",
    "climate", "change", "global", "warming", "carbon", "emission", "greenhouse", "gas", "renewable", "energy",
    "solar", "wind", "hydro", "geothermal", "bioenergy", "recycling", "waste", "management", "pollution", "air",
    "water", "soil", "landfill", "conservation", "area", "national", "park", "protected", "area", "reserve",
    "flora", "fauna", "endangered", "species", "invasive", "species", "pest", "control", "ecosystem", "restoration",
    "reforestation", "afforestation", "deforestation", "land", "management", "agriculture", "permaculture", "organic", "farming", "sustainable",
    "development", "urban", "planning", "green", "space", "building", "design", "efficiency", "water", "saving", "energy",
    "saving", "transport", "cycling", "walking", "public", "transport", "electric", "vehicle", "carbon", "footprint", "offset",
    "green", "technology", "innovation", "research", "education", "awareness", "community", "engagement", "policy", "regulation",
    "legislation", "funding", "investment", "grant", "subsidy", "partnership", "volunteer", "campaign", "advocacy", "movement",
    "clean", "air", "clean", "water", "marine", "conservation", "coral", "reef", "protection", "sustainable", "fishing",
    "aquaculture", "marine", "park", "ocean", "clean", "up", "plastic", "reduction", "reuse", "reduce", "recycle",
    "environmental", "justice", "equity", "indigenous", "rights", "land", "rights", "cultural", "heritage", "traditional", "knowledge"
],
    "Society and Culture": [
    "society", "culture", "community", "heritage", "tradition", "language", "aboriginal", "torres", "strait", "islander",
    "multicultural", "immigration", "diversity", "equality", "inclusion", "reconciliation", "respect", "celebration", "art", "music",
    "dance", "theatre", "literature", "poetry", "storytelling", "film", "cinema", "photography", "painting", "sculpture",
    "gallery", "museum", "festival", "ceremony", "ritual", "custom", "belief", "religion", "spirituality", "ancestry",
    "history", "colonial", "settlement", "exploration", "convict", "migration", "citizenship", "nationality", "identity", "democracy",
    "government", "law", "justice", "freedom", "rights", "welfare", "health", "education", "employment", "economy",
    "environment", "conservation", "sustainability", "urban", "rural", "regional", "coastal", "outback", "landscape", "wildlife",
    "flora", "fauna", "climate", "weather", "beach", "ocean", "reef", "bush", "desert", "mountain",
    "river", "lake", "sport", "recreation", "leisure", "hobby", "volunteer", "charity", "nonprofit", "activism",
    "advocacy", "campaign", "movement", "equality", "justice", "accessibility", "mobility", "technology", "innovation", "research",
    "development", "globalization", "trade", "international", "relations", "peace", "security", "humanitarian", "aid", "cooperation",
    "exchange", "dialogue", "understanding", "partnership", "collaboration", "network", "community", "engagement", "participation", "involvement",
    "contribution", "impact", "change", "progress", "future", "legacy", "memory", "commemoration", "monument", "landmark"
],
    "International Relations and Trade": [
    "international", "relations", "trade", "diplomacy", "treaty", "agreement", "partnership", "alliance", "negotiation", "dialogue",
    "cooperation", "conflict", "resolution", "peace", "security", "defense", "military", "strategy", "policy", "foreign",
    "affairs", "embassy", "consulate", "ambassador", "diplomat", "summit", "conference", "forum", "union", "organization",
    "commission", "delegation", "sanction", "embargo", "tariff", "quota", "trade", "barrier", "export", "import",
    "commodity", "goods", "service", "market", "economy", "investment", "finance", "currency", "exchange", "rate",
    "inflation", "deficit", "surplus", "balance", "payment", "development", "aid", "grant", "loan", "funding",
    "assistance", "humanitarian", "relief", "project", "program", "initiative", "sustainability", "environment", "climate", "change",
    "energy", "resource", "conservation", "biodiversity", "pandemic", "health", "education", "culture", "technology", "innovation",
    "research", "collaboration", "intellectual", "property", "patent", "copyright", "standard", "regulation", "compliance", "ethics",
    "globalization", "multilateral", "bilateral", "regional", "asia", "pacific", "asean", "apec", "g20", "un",
    "nato", "who", "wto", "imf", "world", "bank", "oecd", "unesco", "climate", "accord",
    "kyoto", "paris", "agreement", "migration", "refugee", "visa", "citizenship", "expatriate", "multiculturalism", "diversity"
]
}

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def assign_topic(text, topic_keywords):
    if not text.strip():
        return "Unknown"
    
    vectorizer = TfidfVectorizer()
    
    text_vector = vectorizer.fit_transform([text])
    
    topic_vectors = vectorizer.transform([' '.join(keywords) for keywords in topic_keywords.values()])
    similarity_scores = cosine_similarity(text_vector, topic_vectors)
    
    best_topic_index = similarity_scores.argmax()
    
    best_topic = list(topic_keywords.keys())[best_topic_index]
    return best_topic

In [14]:
df['topic'] = df['pre_processed_text'].apply(lambda x: assign_topic(x, topic_words))


In [ ]:
df